## Configuration

This cell contains all key parameters for analysis. To replicate the paper's results, run this cell and "Run All".

In [ ]:
# Standard Imports
import pandas as pd
import numpy as np
import os
import time
import joblib

# Machine Learning and Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.ensemble import HistGradientBoostingClassifier as HGBoost
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from scipy.stats import pointbiserialr
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import brier_score_loss, make_scorer, average_precision_score, log_loss, matthews_corrcoef, f1_score
from sklearn.base import clone
import statsmodels.api as sm
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import shap


# Google Drive Mounting and Paths
from google.colab import drive
drive.mount('/content/drive')
BASE_PATH = "/content/drive/MyDrive/Diffusion_Indices_Project/"
INTERMEDIATE_PATH = os.path.join(BASE_PATH, "03_intermediate_data")
RESULTS_PATH = os.path.join(BASE_PATH, "04_results")
OOS_PRED_PATH = os.path.join(RESULTS_PATH, "oos_predictions")
SUB_INDICES_PATH = os.path.join(RESULTS_PATH, 'sub_indices_for_tuning')
VISUALS_PATH = os.path.join(BASE_PATH, '05_visuals')
os.makedirs(SUB_INDICES_PATH, exist_ok=True)

# Create the results directories if they don't exist
os.makedirs(RESULTS_PATH, exist_ok=True)
os.makedirs(OOS_PRED_PATH, exist_ok=True)

# Out-of-sample (OOS) Loop Settings
OOS_START_DATE = '1990-01-01'
PREDICTION_HORIZONS = [12]
USE_LAGS = False

ngrid = 30
C_values = np.logspace(-3, 1, ngrid)

# These are the five models used to generate the ensemble forecasts
MODELS_TO_RUN = {
    'Logit': LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000, random_state=42),
    'Logit_L1': LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000, random_state=42),
    'Logit_L2': LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, random_state=42),
    # 'Logit_ElasticNet': LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, max_iter=1000),
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False),
}

# Rerun Control
# Set to True to delete and regenerate all results files.
FORCE_RERUN_ALL_SETS = False
FORCE_RERUN_SPECIFIC_SETS = [] # This is ignored if the RERUN_ALL_SETS is True

## Loading Data

The following steps load data from the original notebook that preprocesses data.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
print("Loading analysis-ready datasets...")
y_target_full = pd.read_pickle(os.path.join(INTERMEDIATE_PATH, 'y_target.pkl'))
X_yield_full = pd.read_pickle(os.path.join(INTERMEDIATE_PATH, 'X_yield.pkl'))
X_transformed_full = pd.read_pickle(os.path.join(INTERMEDIATE_PATH, 'X_transformed_monthly.pkl'))
X_untransformed_full = pd.read_pickle(os.path.join(INTERMEDIATE_PATH, 'X_untransformed_monthly.pkl'))
X_ads_full = pd.read_pickle(os.path.join(INTERMEDIATE_PATH, 'X_ads.pkl'))
tcodes = pd.read_pickle(os.path.join(INTERMEDIATE_PATH, 'tcodes.pkl'))

print("All data loaded successfully.")
print(f"Data shape: {X_transformed_full.shape}, Target shape: {y_target_full.shape}")

In [ ]:
VARS_TO_REMOVE = ['ACOGNO', 'TWEXAFEGSMTHx', 'UMCSENTx', 'OILPRICEx']

vars_that_exist_to_remove = [var for var in VARS_TO_REMOVE if var in X_transformed_full.columns]

X_transformed_full = X_transformed_full.drop(columns=vars_that_exist_to_remove)
X_untransformed_full = X_untransformed_full.drop(columns=vars_that_exist_to_remove)

print(f"--- Data Filtering Complete ---")
print(f"Removed {len(vars_that_exist_to_remove)} problematic variables from all master DataFrames.")
print(f"The list of removed variables is: {vars_that_exist_to_remove}")
print(f"New data shape: {X_transformed_full.shape}")

## Predictor Set Generation

Includes functions for selecting top variables per category, then creating index (two factor, just weakness, just deterioration).

In [ ]:
def generate_PCA_Factors(X_transformed_train, n_factors=8):
    print("Generating PCA Factors...")

    X_stat = X_transformed_train.copy()
    cols_to_drop_nan = X_stat.columns[X_stat.isna().all()]
    if not cols_to_drop_nan.empty:
        print(f"\n Dropping {len(cols_to_drop_nan)} all-NaN columns: {cols_to_drop_nan.to_list()}", end="")
    X_stat_valid = X_stat.drop(columns=cols_to_drop_nan)

    # Imputation on only current slice
    imputer = KNNImputer(n_neighbors=5)
    X_imputed = pd.DataFrame(imputer.fit_transform(X_stat_valid),
                             index=X_stat_valid.index,
                             columns=X_stat_valid.columns)

    # Standardization
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed),
                            index=X_imputed.index,
                            columns=X_imputed.columns)

    # Drop constant columns
    variances = X_scaled.var()
    constant_cols = variances[variances < 1e-10].index
    if not constant_cols.empty:
        print(f"\n         ... Dropping {len(constant_cols)} constant columns: {constant_cols.to_list()}", end="")
    X_final_for_pca = X_scaled.drop(columns=constant_cols)


    pca = PCA(n_components=n_factors, random_state=42)
    factors = pca.fit_transform(X_final_for_pca)

    pca_factors_df = pd.DataFrame(factors,
                                  index=X_final_for_pca.index,
                                  columns=[f'PCA_Factor_{i+1}' for i in range(n_factors)])

    loadings = pca.components_.T
    loadings_df = pd.DataFrame(loadings,
                               index=X_final_for_pca.columns,
                               columns=[f'PCA_Factor_{i+1}' for i in range(n_factors)])

    return pca_factors_df, loadings_df

def generate_PCA_Factors_Binary(X_transformed_train, n_factors=8):
    print("Generating Binary PCA Factors...")

    X_stat = X_transformed_train.copy()
    cols_to_drop_nan = X_stat.columns[X_stat.isna().all()]
    if not cols_to_drop_nan.empty:
        print(f"\n Dropping {len(cols_to_drop_nan)} all-NaN columns: {cols_to_drop_nan.to_list()}", end="")
    X_stat_valid = X_stat.drop(columns=cols_to_drop_nan)

    imputer = KNNImputer(n_neighbors=5)
    X_imputed = pd.DataFrame(imputer.fit_transform(X_stat_valid),
                             index=X_stat_valid.index,
                             columns=X_stat_valid.columns)

    variances = X_imputed.var()
    constant_cols = variances[variances < 1e-10].index
    if not constant_cols.empty:
        print(f"\n Dropping {len(constant_cols)} constant columns: {constant_cols.to_list()}", end="")
    X_final_for_pca = X_imputed.drop(columns=constant_cols)


    pca = PCA(n_components=n_factors, random_state=42)
    factors = pca.fit_transform(X_final_for_pca)

    pca_factors_df = pd.DataFrame(factors,
                                  index=X_final_for_pca.index,
                                  columns=[f'PCA_Factor_{i+1}' for i in range(n_factors)])

    loadings = pca.components_.T
    loadings_df = pd.DataFrame(loadings,
                               index=X_final_for_pca.columns,
                               columns=[f'PCA_Factor_{i+1}' for i in range(n_factors)])

    return pca_factors_df, loadings_df



def generate_Deter_Indices(X_transformed_train, y_train, horizon, threshold=None, counter_threshold=None, var_threshold=None, sector_threshold=None, window_size=None):
    """
    This is the final, unified framework. It uses Ridge (L2) for nowcasting (h<3)
    to retain all signals, and LASSO (L1) for forecasting (h>=3) to perform
    automated feature selection and remove noise.
    Accepts an optional data-driven counter_cyclical_vars list.
    """
    print(f"      -> Generating Deterioration Indices (h={horizon})...")

    variable_groups = {
        'Output_Income': ['RPI', 'W875RX1', 'INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD', 'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S', 'IPFUELS', 'CUMFNS'],
        'Labor_Market': ['HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV', 'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV', 'UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD', 'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP', 'SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT', 'CES0600000007', 'AWOTMAN', 'AWHMAN', 'CES0600000008', 'CES2000000008', 'CES3000000008'],
        'Housing': ['HOUST', 'HOUSTNE', 'HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW', 'PERMITS', 'PERMITW'],
        'Consumption_Orders_Inventories': ['DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx', 'AMDMNOx', 'AMDMUOx', 'ANDENOx', 'BUSINVx', 'ISRATIOx'],
        'Money_Credit': ['M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS', 'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'DTCOLNVHFNM', 'DTCTHFNM', 'INVEST'],
        'Interest_Rates_Spreads': ['FEDFUNDS', 'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA', 'COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM', 'AAAFFM', 'BAAFFM', 'EXSZUSx', 'EXJPUSx', 'EXUSUKx', 'EXCAUSx'],
        'Prices': ['WPSFD49207', 'WPSFD49502', 'WPSID61', 'WPSID62', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL', 'CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD', 'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5', 'PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA'],
        'Stock_Market': ['S&P 500', 'S&P div yield', 'S&P PE ratio', 'VIXCLSx']
    }
    counter_cyclical_vars = {'UNRATE', 'UMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV', 'UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'ISRATIOx', 'VIXCLSx'}

    deterioration_states = pd.DataFrame(index=X_transformed_train.index)


    all_selected_vars = [var for var_list in variable_groups.values() for var in var_list]
    for var in all_selected_vars:
        if var not in X_transformed_train.columns:
            continue
        signal_for_ranking = X_transformed_train[var]
        is_counter_theoretical = var in counter_cyclical_vars
        use_counter_logic = is_counter_theoretical

        if window_size is not None:
            signal_for_ranking = signal_for_ranking.rolling(window=window_size, min_periods=1).mean()
        input_signal = signal_for_ranking

        if threshold is not None:
            quantile = threshold
            if counter_threshold is not None:
                counter_quantile = counter_threshold
            else:
                counter_quantile = 1 - threshold

        if var_threshold is not None:
            quantile = var_threshold.get(var)
            counter_quantile = 1 - quantile

        if sector_threshold is not None:
            var_sector = None
            for sector, vars_list in variable_groups.items():
                if var in vars_list:
                    var_sector = sector
                    break
            quantile = sector_threshold.get(var_sector)
            counter_quantile = 1 - quantile

        deterioration_threshold = input_signal.quantile(counter_quantile if use_counter_logic else quantile)
        deteriorating_state = pd.Series(0.0, index=input_signal.index)
        if use_counter_logic: deteriorating_state[input_signal > deterioration_threshold] = 1
        else: deteriorating_state[input_signal < deterioration_threshold] = 1
        deterioration_states[var] = deteriorating_state

    return deterioration_states

def add_lags(df, lags_to_add, prefix=''):
    """
    Adds lagged versions of columns to a DataFrame.
    """
    if not lags_to_add:
        return df

    df_lagged = df.copy()
    for lag in lags_to_add:
        df_shifted = df.shift(lag)
        df_shifted.columns = [f'{prefix}{col}_lag{lag}' for col in df.columns]
        df_lagged = pd.concat([df_lagged, df_shifted], axis=1)

    return df_lagged

## C Optimization

In [ ]:
# Dr. Shin's C selecting func
def optimize_C_value_first_sample_l1(X_train, y_train, C_values, n_splits=5):
    """
    Optimize C value using time series cross-validation on the first training sample.

    Parameters:
    -----------
    X_train : pd.DataFrame
        Training features
    y_train : pd.Series
        Training target
    C_values : array
        Grid of C values to test
    n_splits : int
        Number of time series splits for cross-validation (default: 7)
        With ~30 years of data, 7 folds gives ~4.3 years per validation set

    Returns:
    --------
    best_C : float
        Best C value based on cross-validation
    """
    if len(X_train) < 50:  # Need sufficient data for cross-validation
        print("      Insufficient data for C optimization, using default C=1.0")
        return 1.0

    tscv = TimeSeriesSplit(n_splits=n_splits)
    best_score = float('inf')
    best_C = 1.0
    c_scores = {}  # Store scores for each C value

    print(f"      Testing {len(C_values)} C values with {n_splits}-fold time series CV...")

    for C in C_values:
        scores = []
        try:
            for train_idx, val_idx in tscv.split(X_train):
                X_train_fold = X_train.iloc[train_idx]
                X_val_fold = X_train.iloc[val_idx]
                y_train_fold = y_train.iloc[train_idx]
                y_val_fold = y_train.iloc[val_idx]

                # Fit model
                model = LogisticRegression(
                    penalty='l1',
                    solver='liblinear',
                    C=C,
                    max_iter=1000,
                    random_state=42,
                    class_weight='balanced'
                )
                model.fit(X_train_fold, y_train_fold)

                # Predict and calculate Brier score
                y_pred_proba = model.predict_proba(X_val_fold)[:, 1]
                brier_score = np.mean((y_val_fold - y_pred_proba) ** 2)
                scores.append(brier_score)

        except Exception as e:
            print(f"      Error with C={C}: {e}")
            continue

        if scores:
            mean_score = np.mean(scores)
            c_scores[C] = mean_score
            if mean_score < best_score:
                best_score = mean_score
                best_C = C

    print(f"      Best C: {best_C:.6f} (Brier Score: {best_score:.6f})")
    print(f"      Tested {len(c_scores)} valid C values")

    # Show top 3 C values for reference
    if len(c_scores) > 3:
        sorted_c = sorted(c_scores.items(), key=lambda x: x[1])[:3]
        print(f"      Top 3 C values: {[(c, score) for c, score in sorted_c]}")

    return best_C

In [ ]:
def optimize_C_value_first_sample_l2(X_train, y_train, C_values, n_splits=5):
    """
    Optimize C value using time series cross-validation on the first training sample.

    Parameters:
    -----------
    X_train : pd.DataFrame
        Training features
    y_train : pd.Series
        Training target
    C_values : array
        Grid of C values to test
    n_splits : int
        Number of time series splits for cross-validation (default: 7)
        With ~30 years of data, 7 folds gives ~4.3 years per validation set

    Returns:
    --------
    best_C : float
        Best C value based on cross-validation
    """
    if len(X_train) < 50:  # Need sufficient data for cross-validation
        print("      Insufficient data for C optimization, using default C=1.0")
        return 1.0

    tscv = TimeSeriesSplit(n_splits=n_splits)
    best_score = float('inf')
    best_C = 1.0
    c_scores = {}  # Store scores for each C value

    print(f"      Testing {len(C_values)} C values with {n_splits}-fold time series CV...")

    for C in C_values:
        scores = []
        try:
            for train_idx, val_idx in tscv.split(X_train):
                X_train_fold = X_train.iloc[train_idx]
                X_val_fold = X_train.iloc[val_idx]
                y_train_fold = y_train.iloc[train_idx]
                y_val_fold = y_train.iloc[val_idx]

                # Fit model
                model = LogisticRegression(
                    penalty='l2',
                    solver='lbfgs',
                    C=C,
                    max_iter=1000,
                    random_state=42,
                    class_weight='balanced'
                )
                model.fit(X_train_fold, y_train_fold)

                # Predict and calculate Brier score
                y_pred_proba = model.predict_proba(X_val_fold)[:, 1]
                brier_score = np.mean((y_val_fold - y_pred_proba) ** 2)
                scores.append(brier_score)

        except Exception as e:
            print(f"      Error with C={C}: {e}")
            continue

        if scores:
            mean_score = np.mean(scores)
            c_scores[C] = mean_score
            if mean_score < best_score:
                best_score = mean_score
                best_C = C

    print(f"      Best C: {best_C:.6f} (Brier Score: {best_score:.6f})")
    print(f"      Tested {len(c_scores)} valid C values")

    # Show top 3 C values for reference
    if len(c_scores) > 3:
        sorted_c = sorted(c_scores.items(), key=lambda x: x[1])[:3]
        print(f"      Top 3 C values: {[(c, score) for c, score in sorted_c]}")

    return best_C

## Optimal Threshold

In [ ]:
def get_empirical_unified_quantile(X_in_sample, y_in_sample, counter_cyclical_vars, horizon, momentum_window):

    print(f"\nDeriving the Empirical Unified Quantile Threshold for h={horizon}")

    if isinstance(y_in_sample, pd.DataFrame):
        y_series = y_in_sample.iloc[:, 0]
    else:
        y_series = y_in_sample

    X_unified = X_in_sample.copy()
    for var in counter_cyclical_vars:
        if var in X_unified.columns:
            X_unified[var] = X_unified[var] * -1

    recession_periods = y_series[y_series == 1].index

    median_recessionary_quantiles = []

    for var in X_unified.columns:
        momentum = X_unified[var].rolling(window=momentum_window).mean()

        rank = momentum.rank(pct=True)

        ranks_during_recession = rank.loc[rank.index.intersection(recession_periods)].dropna()

        if not ranks_during_recession.empty:
            median_rank = ranks_during_recession.median()
            median_recessionary_quantiles.append(median_rank)

    final_empirical_quantile = np.median(median_recessionary_quantiles)

    print(f"Empirically Derived Consensus Quantile Threshold: {final_empirical_quantile:.3f}")

    return final_empirical_quantile

In [ ]:
def get_empirical_variable_quantiles(X_in_sample, y_in_sample, counter_cyclical_vars, horizon):
    """
    Derives the empirical quantile threshold for each individual variable based on its median rank during recessions.
    """

    if isinstance(y_in_sample, pd.DataFrame):
        y_series = y_in_sample.iloc[:, 0]
    else:
        y_series = y_in_sample

    X_unified = X_in_sample.copy()
    for var in counter_cyclical_vars:
        if var in X_unified.columns:
            X_unified[var] = X_unified[var] * -1

    X_ranks = X_unified.rank(pct=True)

    recession_periods = y_in_sample[y_in_sample == 1].index

    ranks_during_recession = X_ranks.loc[X_ranks.index.intersection(recession_periods)].dropna()

    # Calculate the median rank for each variable during recessions
    median_variable_ranks = ranks_during_recession.median()

    variable_quantiles = median_variable_ranks.to_dict()

    print(f"Derived empirical quantiles for {len(variable_quantiles)} variables.")

    return variable_quantiles

In [ ]:
def get_empirical_sector_quantiles(X_in_sample, y_in_sample, variable_groups, counter_cyclical_vars, horizon):
    """
    Derives the empirical quantile threshold for each individual variable based on its median rank during recessions.
    """

    if isinstance(y_in_sample, pd.DataFrame):
        y_series = y_in_sample.iloc[:, 0]
    else:
        y_series = y_in_sample

    X_unified = X_in_sample.copy()
    for var in counter_cyclical_vars:
        if var in X_unified.columns:
            X_unified[var] = X_unified[var] * -1

    X_ranks = X_unified.rank(pct=True)

    recession_periods = y_in_sample[y_in_sample == 1].index

    ranks_during_recession = X_ranks.loc[X_ranks.index.intersection(recession_periods)].dropna()

    # Calculate the median rank for each variable during recessions
    median_variable_ranks = ranks_during_recession.median()

    sector_quantiles = {}
    for sector, var_list in variable_groups.items():
        vars_in_sector = [var for var in var_list if var in median_variable_ranks.index]
        if vars_in_sector:
          sector_quantiles[sector] = median_variable_ranks.loc[vars_in_sector].median()

    return sector_quantiles

## Variables

In [ ]:
variable_groups = {
        'Output_Income': ['RPI', 'W875RX1', 'INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD', 'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S', 'IPFUELS', 'CUMFNS'],
        'Labor_Market': ['HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV', 'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV', 'UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD', 'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP', 'SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT', 'CES0600000007', 'AWOTMAN', 'AWHMAN', 'CES0600000008', 'CES2000000008', 'CES3000000008'],
        'Housing': ['HOUST', 'HOUSTNE', 'HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW', 'PERMITS', 'PERMITW'],
        'Consumption_Orders_Inventories': ['DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx', 'AMDMNOx', 'AMDMUOx', 'ANDENOx', 'BUSINVx', 'ISRATIOx'],
        'Money_Credit': ['M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS', 'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'DTCOLNVHFNM', 'DTCTHFNM', 'INVEST'],
        'Interest_Rates_Spreads': ['FEDFUNDS', 'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA', 'COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM', 'AAAFFM', 'BAAFFM', 'EXSZUSx', 'EXJPUSx', 'EXUSUKx', 'EXCAUSx'],
        'Prices': ['WPSFD49207', 'WPSFD49502', 'WPSID61', 'WPSID62', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL', 'CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD', 'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5', 'PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA'],
        'Stock_Market': ['S&P 500', 'S&P div yield', 'S&P PE ratio', 'VIXCLSx']
}

## Main Forecasting Loop

Parameters for the loop (such as horizons, lags, models, rerun control, etc.) can be found in the config cell.

To add a new predictor set:

---
1. Create generator function in previous section.
2. Add to `ALL_POSSIBLE_SETS` array.
3. Add an `if` block in the predictor set generation section of the loop. Follow format of other sets.
4. Run the loop. It should detect that there is a new predictor set and only run that one, adding it to existing results (unless `FORCE_RERUN_ALL_SETS = True`).




*   To rerun all sets, set `FORCE_RERUN_ALL_SETS` to `True`. This will ignore saved files and generate new results from scratch.
*   To rerun specific predictor set, add it to the `FORCE_RERUN_SPECIFIC_SETS` array.





In [ ]:
print("\nStep 4: Starting recursive out-of-sample forecasting loop...")

SAVE_MODELS = True
OOS_MODELS_PATH = os.path.join(RESULTS_PATH, 'models')

oos_deter_indices_history = []

oos_loadings = {}

unified_thresholds = {}
separate_thresholds = {}
counter_cyclical_vars = {
    'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV',
    'UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'VIXCLSx', 'ISRATIOx'
}

USE_SUBSETS = False
subset = ['PAYEMS', 'CLAIMSx', 'UNRATE', 'AWHMAN', 'INDPRO', 'RPI', 'HOUST', 'RETAILx']


# Master Loop for All Horizons
for PREDICTION_HORIZON in PREDICTION_HORIZONS:
    print(f"\n{'='*25} Processing Horizon h={PREDICTION_HORIZON} {'='*25}")

    X_transformed_shifted = X_transformed_full.shift(PREDICTION_HORIZON)
    X_untransformed_shifted = X_untransformed_full.shift(PREDICTION_HORIZON)
    X_yield_shifted = X_yield_full.shift(PREDICTION_HORIZON)
    X_ads_shifted = X_ads_full.shift(PREDICTION_HORIZON)

    if USE_LAGS:
        LAGS_TO_ADD = [3, 6, 12]
    else:
        LAGS_TO_ADD = []

    if LAGS_TO_ADD:
      file_path = os.path.join(OOS_PRED_PATH, f'oos_results_h{PREDICTION_HORIZON}_final_fixlagged_bestpred.pkl')
    else:
      file_path = os.path.join(OOS_PRED_PATH, f'oos_results_h{PREDICTION_HORIZON}_final_marx_maf.pkl')

    horizon_sub_index_path = os.path.join(SUB_INDICES_PATH, f'h{PREDICTION_HORIZON}')
    os.makedirs(horizon_sub_index_path, exist_ok=True)

    if SAVE_MODELS:
        horizon_model_path = os.path.join(OOS_MODELS_PATH, f'h{PREDICTION_HORIZON}')
        os.makedirs(horizon_model_path, exist_ok=True)
        print(f"Models for h = {PREDICTION_HORIZON} will be saved to: {horizon_model_path}")


    oos_loadings[PREDICTION_HORIZON] = {
        'PCA_Factors_8': {},
        'Deter_PCA': {},
        'Deter_PCA_SecQ': {},
        'Deter_PCA_VarQ': {},
    }
    # Load existing results or initialize new ones
    oos_probs, oos_errors, oos_actuals, oos_importances = {}, {}, None, {} # Start with empty dicts
    if not FORCE_RERUN_ALL_SETS:
        try:
            print(f"Attempting to load existing results from: {file_path}")
            oos_results = joblib.load(file_path)
            oos_probs = oos_results.get('probabilities', {}) # Use .get for safety
            oos_errors = oos_results.get('squared_errors', {})
            oos_actuals = oos_results.get('actuals', None)
            oos_importances = oos_results.get('importances', {})
            print("Successfully loaded existing results.")
        except FileNotFoundError:
            print("No existing results file found. Initializing new structure.")
    else:
        print(f"FORCE_RERUN_ALL_SETS is True. Any loaded results will be ignored for this horizon.")

    # Determine which predictor sets need to be run
    ALL_POSSIBLE_SETS = ['MARX', 'MAF']


    sets_to_run = []
    if FORCE_RERUN_ALL_SETS:
        sets_to_run = ALL_POSSIBLE_SETS
        print(f"All {len(sets_to_run)} predictor sets will be re-run.")
    else:
        for pred_set in ALL_POSSIBLE_SETS:
            # Rerun if not found or if specifically requested
            if pred_set not in oos_probs or pred_set in FORCE_RERUN_SPECIFIC_SETS:
                sets_to_run.append(pred_set)
        if FORCE_RERUN_SPECIFIC_SETS:
             print(f"Will force rerun for specific sets: {FORCE_RERUN_SPECIFIC_SETS}")

    if not sets_to_run:
        print("All specified predictor sets have already been run for this horizon. Skipping.")
        continue

    print(f"The following {len(sets_to_run)} sets will be run: {sets_to_run}")

    # Initialize/clear storage ONLY for the sets that are being run
    for pred_set in sets_to_run:
        oos_probs[pred_set] = {m: [] for m in MODELS_TO_RUN}
        oos_errors[pred_set] = {m: [] for m in MODELS_TO_RUN}
        oos_importances[pred_set] = {m: [] for m in MODELS_TO_RUN}


    target_col_name = 'USRECM'

    # Main time-series loop
    all_dates = y_target_full.index
    forecast_dates = all_dates[all_dates >= pd.to_datetime(OOS_START_DATE)]

    if oos_actuals is None or (len(oos_actuals) != len(forecast_dates)):
        oos_actuals = y_target_full.loc[forecast_dates, target_col_name]
        oos_actuals.index.name = 'Date'


    start_time = time.time()

    optimal_C_values_l1 = {}
    optimal_C_values_l2 = {}

    oos_training_datasets = {pred_set: [] for pred_set in sets_to_run}


    for i, forecast_date in enumerate(forecast_dates):
        iter_start_time = time.time()
        train_end_date = forecast_date - pd.DateOffset(months=PREDICTION_HORIZON)

        y_train_full = y_target_full.loc[:train_end_date, target_col_name]
        y_actual = oos_actuals.loc[forecast_date]

        print(f"Iter {i+1}/{len(forecast_dates)}: h={PREDICTION_HORIZON}, Date={forecast_date.date()}... ", end="")


        # Centralized Data Preparation
        X_train_transformed_slice = X_transformed_shifted.loc[:forecast_date].copy()
        if USE_SUBSETS:
            X_train_transformed_slice = X_train_transformed_slice[subset]
        X_untransformed_slice = X_untransformed_shifted.loc[:forecast_date].copy()

        X_train_valid_cols = X_train_transformed_slice.drop(columns=X_train_transformed_slice.columns[X_untransformed_slice.isna().all()]).copy()
        imputer_base = KNNImputer(n_neighbors=5)
        X_train_imputed = pd.DataFrame(imputer_base.fit_transform(X_train_valid_cols), index=X_train_valid_cols.index, columns=X_train_valid_cols.columns)
        scaler = StandardScaler()
        X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_imputed), index=X_train_imputed.index, columns=X_train_imputed.columns)

        # Quantile threshold tuning
        first_forecast_date = pd.to_datetime(OOS_START_DATE)
        tuning_train_end_date = first_forecast_date - pd.DateOffset(months=PREDICTION_HORIZON)

        X_in_sample_for_tuning = X_transformed_full.loc[:tuning_train_end_date]
        if USE_SUBSETS:
            X_in_sample_for_tuning = X_in_sample_for_tuning[subset]
        y_in_sample_for_tuning = y_target_full.loc[:tuning_train_end_date, 'USRECM']

        optimal_q = get_empirical_unified_quantile(X_in_sample_for_tuning, y_in_sample_for_tuning, counter_cyclical_vars, horizon=PREDICTION_HORIZON, momentum_window=12)
        optimal_var_q = get_empirical_variable_quantiles(X_in_sample_for_tuning, y_in_sample_for_tuning, counter_cyclical_vars, horizon=PREDICTION_HORIZON)
        optimal_sec_q = get_empirical_sector_quantiles(X_in_sample_for_tuning, y_in_sample_for_tuning, variable_groups, counter_cyclical_vars, horizon=PREDICTION_HORIZON)


        if i == 0:
          print(f"Unified Quantile Threshold for h={PREDICTION_HORIZON}: {optimal_q}")
          print("="*70 + "\n")
          print(f"Variable Quantile Thresholds for h={PREDICTION_HORIZON}: {optimal_var_q}")
          print("="*70 + "\n")
          print(f"Sector Quantile Thresholds for h={PREDICTION_HORIZON}: {optimal_sec_q}")
          print("="*70 + "\n")

        # Generate Predictor Sets
        predictor_data_iter = {}


        # if 'ADS' in sets_to_run:
        #     predictor_data_iter['ADS'] = X_ads_shifted.loc[:forecast_date]
        # if 'Yield' in sets_to_run:
        #     predictor_data_iter['Yield'] = X_yield_shifted.loc[:forecast_date]
        if 'Full' in sets_to_run: predictor_data_iter['Full'] = X_train_scaled
        if 'PCA_Factors_8' in sets_to_run:
            all_8_factors, loadings = generate_PCA_Factors(X_train_transformed_slice, n_factors=8)
            predictor_data_iter['PCA_Factors_8'] = all_8_factors
            oos_loadings[PREDICTION_HORIZON]['PCA_Factors_8'][forecast_date] = loadings

        states_global_q = None
        if any(s in ['Deter_States', 'Deter_PCA', 'Deter_Avg', 'Deter_States_BestPred'] for s in sets_to_run):
            print("Generating binary states with GLOBAL threshold...")
            states_global_q = generate_Deter_Indices(X_train_imputed, y_train_full, horizon=PREDICTION_HORIZON, threshold=optimal_q)

        states_sector_q = None
        if any(s in ['Deter_States_SecQ', 'Deter_PCA_SecQ', 'Deter_Avg_SecQ'] for s in sets_to_run):
            print("Generating binary states with SECTOR thresholds...")
            states_sector_q = generate_Deter_Indices(X_train_imputed, y_train_full, horizon=PREDICTION_HORIZON, sector_threshold=optimal_sec_q)

        states_variable_q = None
        if any(s in ['Deter_States_VarQ', 'Deter_PCA_VarQ', 'Deter_Avg_VarQ'] for s in sets_to_run):
            print("Generating binary states with VARIABLE thresholds...")
            states_variable_q = generate_Deter_Indices(X_train_imputed, y_train_full, horizon=PREDICTION_HORIZON, var_threshold=optimal_var_q)

        if 'Deter_States' in sets_to_run:
            predictor_data_iter['Deter_States'] = states_global_q
        if 'Deter_States_SecQ' in sets_to_run:
            predictor_data_iter['Deter_States_SecQ'] = states_sector_q
        if 'Deter_States_VarQ' in sets_to_run:
            predictor_data_iter['Deter_States_VarQ'] = states_variable_q

        if 'Deter_PCA' in sets_to_run:
            predictor_data_iter['Deter_PCA'], loadings = generate_PCA_Factors_Binary(states_global_q, n_factors=8)
            oos_loadings[PREDICTION_HORIZON]['Deter_PCA'][forecast_date] = loadings
        if 'Deter_PCA_SecQ' in sets_to_run:
            predictor_data_iter['Deter_PCA_SecQ'], loadings = generate_PCA_Factors_Binary(states_sector_q, n_factors=8)
            oos_loadings[PREDICTION_HORIZON]['Deter_PCA_SecQ'][forecast_date] = loadings
        if 'Deter_PCA_VarQ' in sets_to_run:
            predictor_data_iter['Deter_PCA_VarQ'], loadings = generate_PCA_Factors_Binary(states_variable_q, n_factors=8)
            oos_loadings[PREDICTION_HORIZON]['Deter_PCA_VarQ'][forecast_date] = loadings

        if 'Deter_Avg' in sets_to_run:
            predictor_data_iter['Deter_Avg'] = states_global_q.mean(axis=1).to_frame(name='Deter_Avg')
        if 'Deter_Avg_SecQ' in sets_to_run:
            predictor_data_iter['Deter_Avg_SecQ'] = states_sector_q.mean(axis=1).to_frame(name='Deter_Avg_SecQ')
        if 'Deter_Avg_VarQ' in sets_to_run:
            predictor_data_iter['Deter_Avg_VarQ'] = states_variable_q.mean(axis=1).to_frame(name='Deter_Avg_VarQ')

        if 'Full_BestPred' in sets_to_run:
            predictor_data_iter['Full_BestPred'] = X_train_scaled
        if 'Deter_States_BestPred' in sets_to_run:
            predictor_data_iter['Deter_States_BestPred'] = states_global_q


        if i == 0:
            print(f"\n*** OPTIMIZING C VALUES ON FIRST TRAINING SAMPLE ***")
            print(f"Date: {forecast_date.date()}, Training data up to: {train_end_date.date()}")
            print(f"Using {len(C_values)} C values from {C_values[0]:.6f} to {C_values[-1]:.6f}")
            print(f"Will optimize C separately for each of the {len(sets_to_run)} predictor sets: {sets_to_run}")

            for pred_set_name in sets_to_run:
                print(f"\n  --- Processing {pred_set_name} ---")
                X_train_raw = predictor_data_iter.get(pred_set_name)

                if X_train_raw is None or X_train_raw.empty:
                    print(f"    No data available, using default C=1.0")
                    optimal_C_values_l1[pred_set_name] = 1.0
                    optimal_C_values_l2[pred_set_name] = 1.0
                    continue

                # Add lags if specified
                X_train_lagged = add_lags(X_train_raw, LAGS_TO_ADD)
                X_train_lagged_final = X_train_lagged.dropna()
                X_train_final = X_train_lagged_final.loc[:train_end_date]

                # Align with target
                common_index = y_train_full.index.intersection(X_train_final.index)
                y_train_aligned = y_train_full.loc[common_index]
                X_train_aligned = X_train_final.loc[common_index]

                print(f"    Data shape: {X_train_aligned.shape}, Target samples: {len(y_train_aligned)}")

                if len(X_train_aligned) > 50:  # Sufficient data for optimization
                    print(f"    Optimizing C value using time series CV...")
                    # Note: C optimization still uses the full training slice available at iter 0
                    optimal_C_l1 = optimize_C_value_first_sample_l1(X_train_aligned, y_train_aligned, C_values)
                    optimal_C_values_l1[pred_set_name] = optimal_C_l1

                    optimal_C_l2 = optimize_C_value_first_sample_l2(X_train_aligned, y_train_aligned, C_values)
                    optimal_C_values_l2[pred_set_name] = optimal_C_l2
                    print(f"    Optimal C (L1) for {pred_set_name}: {optimal_C_l1:.6f}")
                    print(f"    Optimal C (L2) for {pred_set_name}: {optimal_C_l2:.6f}")
                else:
                    print(f"    Insufficient data ({len(X_train_aligned)} samples), using default C=1.0")
                    optimal_C_values_l1[pred_set_name] = 1.0
                    optimal_C_values_l2[pred_set_name] = 1.0

            print(f"\n*** C OPTIMIZATION COMPLETE ***")
            print("Summary of optimal C (L1) values by predictor set:")
            for pred_set, opt_c in optimal_C_values_l1.items():
                print(f"  {pred_set:20s}: C = {opt_c:.6f}")
            print("Summary of optimal C (L2) values by predictor set:")
            for pred_set, opt_c in optimal_C_values_l2.items():
                print(f"  {pred_set:20s}: C = {opt_c:.6f}")


        # Loop over the INTENDED sets
        for pred_set_name in sets_to_run:
            X_train_raw = predictor_data_iter.get(pred_set_name)

            X_train_saved = X_train_raw.loc[:train_end_date]
            oos_training_datasets[pred_set_name].append(X_train_saved)

            if X_train_raw is None or X_train_raw.empty:
                for model_name in MODELS_TO_RUN:
                    oos_probs[pred_set_name][model_name].append(np.nan)
                    oos_errors[pred_set_name][model_name].append(np.nan)
                continue
            else:
                X_train_lagged = add_lags(X_train_raw, LAGS_TO_ADD)

            X_train_lagged_final = X_train_lagged.dropna()
            X_train_final = X_train_lagged_final.loc[:train_end_date]

            X_predict_point = X_train_lagged_final.loc[[forecast_date]]


            for model_name, model_template in MODELS_TO_RUN.items():
                prob, error = np.nan, np.nan
                try:
                    common_index = y_train_full.index.intersection(X_train_final.index)
                    y_train_aligned = y_train_full.loc[common_index]
                    X_train_aligned = X_train_final.loc[common_index]

                    if len(X_train_aligned) > max(LAGS_TO_ADD, default=0) + 20:
                        model_instance = clone(model_template)
                        params_to_set = {}

                        if 'XGBoost' in model_name:
                            neg, pos = (y_train_aligned == 0).sum(), (y_train_aligned == 1).sum()
                            if pos > 0: params_to_set['scale_pos_weight'] = (neg / pos)
                        elif 'HGBoost' in model_name or 'RandomForest' in model_name or 'Logit' in model_name:
                            params_to_set['class_weight'] = 'balanced'

                        if 'Logit_L1' in model_name and pred_set_name in optimal_C_values_l1:
                            params_to_set['C'] = optimal_C_values_l1[pred_set_name]


                        if 'Logit_L2' in model_name and pred_set_name in optimal_C_values_l2:
                            params_to_set['C'] = optimal_C_values_l2[pred_set_name]

                        if params_to_set:
                            model_instance = model_instance.set_params(**params_to_set)

                        X_predict_imputed = X_predict_point.reindex(columns=X_train_aligned.columns).ffill().bfill()

                        if not X_predict_imputed.isna().any().any():
                            model_instance.fit(X_train_aligned, y_train_aligned)

                            try:
                                if 'XGBoost' in model_name:
                                    booster = model_instance.get_booster()
                                    importance_dict = booster.get_score(importance_type='gain')
                                    importances = pd.Series(importance_dict, index=X_train_aligned.columns).fillna(0)
                                elif 'Logit' in model_name:
                                    importances = pd.Series(np.abs(model_instance.coef_[0]), index=X_train_aligned.columns)
                                else: # RandomForest, HGBoost
                                    importances = pd.Series(model_instance.feature_importances_, index=X_train_aligned.columns)


                            except Exception as e_imp:
                                print(f"Could not get importances for {model_name} on {forecast_date.date()}: {e_imp}")
                                pass

                            if 'SAVE_MODELS' and forecast_date == forecast_dates[-1]:
                                print("Last iteration. Saving model...")
                                if not LAGS_TO_ADD:
                                    model_filename = f'{pred_set_name}_{model_name}_model.pkl'
                                    traindata_filename = f'{pred_set_name}_{model_name}_traindata.pkl'
                                else:
                                    model_filename = f'{pred_set_name}_{model_name}_lagged_model.pkl'
                                    traindata_filename = f'{pred_set_name}_{model_name}_lagged_traindata.pkl'

                                model_path = os.path.join(horizon_model_path, model_filename)
                                joblib.dump(model_instance, model_path)

                                traindata_path = os.path.join(horizon_model_path, traindata_filename)
                                X_train_aligned.to_pickle(traindata_path)

                                if pred_set_name in ['PCA_Factors_8', 'Deter_PCA']:
                                    loadings_for_model = oos_loadings[PREDICTION_HORIZON][pred_set_name][forecast_date]
                                    loadings_filename = f'{pred_set_name}_{model_name}_loadings.pkl'
                                    loadings_path = os.path.join(horizon_model_path, loadings_filename)
                                    loadings_for_model.to_pickle(loadings_path)

                            prob = model_instance.predict_proba(X_predict_imputed)[:, 1][0]
                            error = (y_actual - prob)**2


                except Exception as e:
                    print(f"Error in model {model_name} for set {pred_set_name}: {e}")
                    pass

                oos_probs[pred_set_name][model_name].append(prob)
                oos_errors[pred_set_name][model_name].append(error)
                oos_importances[pred_set_name][model_name].append(importances)

        iter_end_time = time.time()
        print(f" ({(iter_end_time - iter_start_time):.2f}s)")

    # Save results after each horizon's loop is complete
    print(f"\n--- Loop for h={PREDICTION_HORIZON} Finished ---")
    results_to_save = {
        'probabilities': oos_probs,
        'squared_errors': oos_errors,
        'actuals': oos_actuals,
        'importances': oos_importances,
        'loadings': oos_loadings,
        'training_datasets': oos_training_datasets
    }
    joblib.dump(results_to_save, file_path)
    print(f"Updated results saved to: {file_path}")


# %% Report results
end_time = time.time()
print(f"Total computation time: {end_time - start_time:.2f} seconds")
print("")

print("\n--- All Horizons Complete ---")